Countermeasures simulations
-----------------------------------

RNN-based approach to simulate the effect of timing and taking action on deaths-per-100k inhabitants.

In [ ]:
import datetime
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

In [ ]:
pd.options.display.max_colwidth = 200
pd.options.display.max_columns = 100

#### Data sources

* The countermeasures data is made available as part of the John Hopkins [Containment Measures Database](http://epidemicforecasting.org/containment).
* The Oxford COVID-19 Government Response Tracker is made available as part of the [OxCGRT](https://www.bsg.ox.ac.uk/research/research-projects/oxford-covid-19-government-response-tracker) project.
* The country population data was gathered from the [Word Bank Databank](https://databank.worldbank.org/reports.aspx?source=2&series=SP.POP.TOTL&country=#), with latest data available for 2018.

In [ ]:
johnshopkins_df = pd.read_csv("data/countermeasures_db_johnshopkins_2020_03_30.csv")
johnshopkins_df["Date"] = johnshopkins_df["Date"].apply(lambda x: pd.to_datetime(x, format="%Y-%m-%d")).dt.date

In [ ]:
oxford_grt_df = pd.read_csv("data/oxford_uni_response_tracker_08042020.csv")
oxford_grt_df["Date"] = oxford_grt_df["Date"].apply(lambda x: pd.to_datetime(str(x), format="%Y-%m-%d")).dt.date

In [ ]:
grt_df = oxford_grt_df.drop_duplicates(subset=["CountryName", "StringencyIndex"], keep="first")

In [ ]:
measures_df = pd.merge(johnshopkins_df, grt_df, how="outer", left_on=["Date", "Country"], 
                       right_on=["Date", "CountryName"])

In [ ]:
population_df = pd.read_csv("data/wb_country_populations_2018.csv")[["Country Name", "2018 [YR2018]"]]
population_df.rename(columns={"2018 [YR2018]": "2018_Country_Population"}, inplace=True)

In [ ]:
base_df = pd.merge(measures_df, population_df, how="outer", left_on="Country", right_on="Country Name").fillna(0)
base_df = base_df.loc[base_df["2018_Country_Population"] != ".."]
base_df = base_df.loc[base_df["Country"] != 0]

In [ ]:
base_df.head()

In [ ]:
print(base_df["Country"].unique())

In [ ]:
base_df.columns

In [ ]:
other_cm_cols = ['Unnamed: 0', 'Resumption', 'Diagnostic criteria tightened', 'Diagnostic criteria loosened',
                 'Testing criteria', 'Date', 'Country', 'Confirmed Cases', 'Deaths', 'Country Name', 'CountryName',
                 'CountryCode', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex', 'StringencyIndexForDisplay',
                 'Unnamed: 39', 'Country Name', '2018_Country_Population', 'Deaths_per_100k']

In [ ]:
countermeasures = list(filter(lambda m: m not in other_cm_cols and not 
                              m.endswith("_Notes") and not m.endswith("_IsGeneral"), base_df.columns))

In [ ]:
df = base_df[countermeasures + ["Date", "Country", "Confirmed Cases", "Deaths", "2018_Country_Population"]].fillna(0)

In [ ]:
df[countermeasures] = df[countermeasures].mask(df[countermeasures] > 0, 1.0)

## Simulation of the effect and timing of countermeasures

Exploration of the impact of government countermeasures on reported Deaths per 100.000 citizens in a single country:

* The [model weights](#Simulation-model) are learnt from the relation between countermeasures and mortality for all the other countries in the dataset.
* The [counterfactual intervention](#Intervention) (the package of measures described below) is compared to the actual mortality rates.

At the moment, the simulation doesn't yet take into account any factors other than the countermeasures (e.g. GDP, Healthcare efficiency, population age, population density) that will also play an important part in determining the morbidity in a country.

#### Simulation parameters

In [ ]:
country = "France"
package = ["Nonessential business suspension", "School closure", "Gatherings banned"]
treatment_start = 21  # Counterfactual start date of intervention in days after first reported case

#### Model parameters

In [ ]:
rnn_params = {
    "sequence_length": 4,
    "batch_size": 32,
    "num_epochs": 50,
    "cell_units": 64,
    "mlp_cells": 16,
    "dropout_rate": 0.25
}

In [ ]:
train_validation_split = 0.85

#### Preprocess the simulation data

In the simulation data, the counterfactual treatment condition is coded '1.0' starting $n$ days after the first reported case of COVID-19 in the country.

In [ ]:
def create_simulation_data(df, country, simulated_measures, treatment_start, measures, seq_length):
    
    # Simulation parameters
    sim_df = df[df["Country"] == country].sort_values(by="Date")
    date_first_reported_case = sim_df.loc[sim_df["Confirmed Cases"].ne(0.0).idxmax():, ["Date"]].values[0][0]
    counterfactual_start_date = date_first_reported_case + datetime.timedelta(days=treatment_start)
    
    # Simulation counterfactuals
    for countermeasure in simulated_measures:
        actual_start_date = sim_df.loc[sim_df[countermeasure].idxmax(1.0):, ["Date"]].values[0][0]
        difference_in_days = (actual_start_date - counterfactual_start_date).days
        sim_df.loc[sim_df.index[sim_df["Date"] == counterfactual_start_date].values[0]:, countermeasure] = 1.0
        
        print(f"Setting counterfactual start date for treatment '{countermeasure}' to {str(counterfactual_start_date)}.")
        print(f"Actual start date of treatment '{countermeasure}': {str(actual_start_date)}.")
        print(f"Intervention in simulation occurs {difference_in_days} days prior to intervention in real life.")
        print("\n")
        
    sim_df[["Reported Deaths", "Reported Cases"]] = sim_df[["Deaths", "Confirmed Cases"]].diff()
    sim_df["Mortality_per_100k"] = sim_df["Reported Deaths"] / (
        sim_df["2018_Country_Population"].astype(float) / 100000.0)
    sim_df["Cases_per_100k"] = sim_df["Reported Cases"] / (
        sim_df["2018_Country_Population"].astype(float) / 100000.0)
    sim_df = sim_df.fillna(0)
    sim_df["DateIndex"] = sim_df["Date"]
    sim_df = sim_df.set_index("DateIndex")
    
    # Prepare data for model input
    sim_label_df = sim_df["Mortality_per_100k"]
    sim_features_df = sim_df[measures + ["Cases_per_100k"]]
    
    sim_features = []
    sim_labels = []

    for i in range(int(len(sim_df) - seq_length)):
        sim_labels.append(sim_label_df[i + seq_length])
        sim_features.append(sim_features_df[i: (i + seq_length)].values)
    
    X = np.asarray(sim_features)
    y = np.asarray(sim_labels)
    y = y.reshape((y.shape[0], 1))
    
    return sim_df, X, y, counterfactual_start_date

#### Intervention

In [ ]:
simulation_df, sim_X, sim_y, cf_start_date = create_simulation_data(df, country, package, treatment_start, 
                                                                    countermeasures, rnn_params["sequence_length"])

In [ ]:
plt.figure(figsize=(16,4))
sns.lineplot(x=simulation_df.index, y="Reported Deaths", data=simulation_df)

In [ ]:
record_start = simulation_df["Date"].head(1).values[0]
record_end = simulation_df["Date"].tail(1).values[0]
num_days = (record_end - record_start).days
num_deaths = simulation_df["Deaths"].tail(1).values[0]
print(f"Recorded {num_deaths} COVID-19 deaths from {str(record_start)} to {str(record_end)} ({num_days} days) in the factual data for {country}.")

#### Preprocess the training and validation data

In the training data, actual observed mortality rates for COVID-19 all countries in the dataset is prepared as the dependent variable ($y$), with the countermeasures taken by national governments making up the features ($X$).

In [ ]:
def create_train_and_validation_data(df, country, train_split, measures, seq_length):
    train_df = df[df["Country"] != country]
    train_countries = list(filter(lambda c: c != country, df["Country"].unique()))
    
    features = []
    labels = []
    
    # Retrieve the feature sequences and mortality data for each of the countries in the dataset
    for country in train_countries:
        country_df = train_df[train_df["Country"] == country].sort_values(by="Date")
        country_df[["Reported Deaths", "Reported Cases"]] = country_df[["Deaths", "Confirmed Cases"]].diff()
        country_df["Mortality_per_100k"] = country_df["Reported Deaths"] / (
            country_df["2018_Country_Population"].astype(float) / 100000.0)
        country_df["Cases_per_100k"] = country_df["Reported Cases"] / (
            country_df["2018_Country_Population"].astype(float) / 100000.0)
        country_df = country_df.replace([-np.inf, np.inf], np.nan)
        country_df = country_df.fillna(0)
        country_df = country_df.set_index("Date")
        
        country_label_df = country_df["Mortality_per_100k"]
        
        country_label_df.loc[country_label_df < 0.0] = 0.0
        country_features_df = country_df[measures + ["Cases_per_100k"]]
        
        for i in range(int(len(country_df) - seq_length)):
            labels.append(country_label_df[i + seq_length])
            features.append(country_features_df[i: (i + seq_length)].values)
    
    # Shuffle the data
    split_point = int(len(features) * train_split)
    data = list(zip(features, labels))
    random.shuffle(data)
    s_features, s_labels = zip(*data)
    
    # Prepare the data format for the model
    train_X = np.asarray(s_features[0:split_point])
    train_y = np.asarray(s_labels[0:split_point])
    train_y = train_y.reshape((train_y.shape[0], 1))

    val_X = np.asarray(s_features[split_point:len(features)])
    val_y = np.asarray(s_labels[split_point:len(labels)])
    val_y = val_y.reshape((val_y.shape[0], 1))
    
    return train_X, train_y, val_X, val_y

In [ ]:
X, y, validation_X, validation_y = create_train_and_validation_data(df, country, train_validation_split, 
                                                                    countermeasures, rnn_params["sequence_length"])

In [ ]:
print(f"Prepared {len(X)} training and {len(validation_X)} validation samples with feature dimensions {X.shape}.")

In [ ]:
pd.Series(y.T[0]).describe()

In [ ]:
pd.Series(validation_y.T[0]).describe()

### Simulation model

In [ ]:
model = tf.keras.models.Sequential(name="simulation_model")
model.add(tf.keras.layers.LSTM(rnn_params["cell_units"], bias_initializer='zeros',
          input_shape=(rnn_params["sequence_length"], len(countermeasures) + 1)))
model.add(tf.keras.layers.Dropout(rnn_params["dropout_rate"]))
model.add(tf.keras.layers.Dense(rnn_params["mlp_cells"], activation="relu"))
model.add(tf.keras.layers.Dropout(rnn_params["dropout_rate"]))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model.compile(loss="mse", optimizer="adam", metrics=["mse", "mae"])

In [ ]:
model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_mae", mode="min", patience=2)

#### Train the model

In [ ]:
train = model.fit(X, y, epochs=rnn_params["num_epochs"], batch_size=rnn_params["batch_size"],
                  callbacks=[early_stopping], validation_data=(validation_X, validation_y))

In [ ]:
plt.plot(train.history["loss"], label="MSE train loss")
plt.plot(train.history["val_loss"], label="MSE validation loss")
plt.legend()
plt.show()

### Run the country simulation with the learnt RNN parameters

In [ ]:
predicted_mortality = model.predict(sim_X)

In [ ]:
pd.Series(predicted_mortality.T[0]).describe()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(sim_y.T[0], label = "observed mortality")
plt.plot(predicted_mortality, label = "counterfactual mortality")
plt.legend()
plt.title(f"Observed vs counterfactual mortality rates in {country} with package <{', '.join(package)}> starting on {cf_start_date}.")
plt.show()